In [1]:
import tensorflow as tf
import numpy as np

/home/bill/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# NER模型: 命名实体识别
input: window  
loss: softmax-CE

## build the graph

In [2]:
#config
n_window_features = 2
embed_size = 10
h_size =3
n_classes = 3
learning_rate = .01

g = tf.Graph()
with g.as_default():
    #parameters
    _init_xavier = tf.contrib.layers.xavier_initializer()
    _init_zero = tf.zeros_initializer()
    W = tf.get_variable('W', shape=(n_window_features*embed_size, h_size), 
                        dtype=tf.float32,
                        initializer=_init_xavier)
    b1 = tf.get_variable('b1', shape=(h_size,),
                       dtype=tf.float32,
                       initializer=_init_zero)
    U = tf.get_variable('U', shape=(h_size, n_classes), 
                        dtype=tf.float32, initializer=_init_xavier)
    b2 = tf.get_variable('b2', shape=(n_classes,),
                         dtype=tf.float32, initializer=_init_zero)
    pretrained_embeddings = tf.constant(np.random.randn(10,10), dtype=tf.float32)
    E = tf.Variable(pretrained_embeddings)
    
    #predition
    x = tf.placeholder(dtype=tf.int32, shape=(None, n_window_features), name='input')
    _e = tf.nn.embedding_lookup(E, x)
    e = tf.reshape(_e, (-1, n_window_features*embed_size))
    h = tf.nn.relu(tf.matmul(e, W) + b1)
    dropout = tf.placeholder(tf.float32, name='dropout')
    h_drop = tf.nn.dropout(h, dropout)
    logits = tf.matmul(h_drop, U) + b2
    labels = tf.placeholder(tf.int32, shape=(None,), name='labels')
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels,
                                                         logits=logits)
    loss = tf.reduce_mean(loss)
    #train
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_op = optimizer.minimize(loss)
    
    #init variables
    init_op = tf.global_variables_initializer()
    saver = tf.train.Saver()

## run the graphical model

In [7]:
#run
# x是一个中心词的window
feed_dict = {x: [[1, 2], [3, 2], [4, 3], [4, 4]],
         labels: [1, 1, 2, 2],
         dropout: .5}

#
model_saved = './tensorflow-save/'
with tf.Session(graph=g) as sess:
    sess.run(init_op)
    N_run = 0
    while True:
        _, _loss = sess.run([train_op,loss],feed_dict=feed_dict)
        N_run += 1
        if N_run%100 == 0:
            print(f'{N_run}的loss为{_loss}')
        if (_loss < .1) or (N_run > 1000):
            print(_loss)
            saver.save(sess, model_saved)
            sess.close()
            break

0.011966233


# restore and predict

In [8]:
feed_dict = {x: [[1, 2], [3, 2], [4, 3], [4, 4]], dropout: 1}
with tf.Session(graph=g) as sess:
    sess.run(init_op)
    saver.restore(sess, model_saved)
    _preds = sess.run(tf.argmax(logits, axis=1), feed_dict=feed_dict)
_preds

INFO:tensorflow:Restoring parameters from ./tensorflow-save/


array([1, 1, 2, 2])